# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,2.83,51,97,2.24,CA,1712096022
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,2.33,82,0,6.26,RU,1712096151
2,2,darwin,-12.4611,130.8418,26.08,93,75,2.57,AU,1712095782
3,3,atafu village,-8.5421,-172.5159,29.21,72,61,1.47,TK,1712096141
4,4,olonkinbyen,70.9221,-8.7187,-3.93,63,73,6.20,SJ,1712096143


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map

map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_df = city_data_df
mask = (temp_df['Max Temp'] <= 30) & (temp_df['Max Temp'] >= 18) & (temp_df['Humidity'] < 75) & (temp_df['Humidity'] > 30) & (temp_df['Cloudiness'] <= 20) & (temp_df['Wind Speed'] <= 4)
#mask = (temp_df['Max Temp'] <= 30) & (temp_df['Max Temp'] >= 18)
ideal_weather_df = temp_df[mask]

ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,88,pacific grove,36.6177,-121.9166,22.03,73,0,3.09,US,1712096204
105,105,newman,37.3138,-121.0208,24.60,41,0,2.68,US,1712096288
119,119,portland,45.5234,-122.6762,24.46,41,20,2.06,US,1712095790
185,185,kerikeri,-35.2268,173.9474,19.96,55,5,0.50,NZ,1712096306
223,223,east london,-33.0153,27.9116,19.04,69,11,3.40,ZA,1712096108
245,245,kone,-21.0595,164.8658,27.43,62,7,2.28,NC,1712096320
315,315,ha'il,27.5219,41.6907,19.01,34,0,1.03,SA,1712096336
324,324,yeroham,30.9872,34.9314,20.38,51,0,2.20,IL,1712096338
376,376,thames,-37.1333,175.5333,19.03,55,4,0.29,NZ,1712096352
483,483,paulden,34.8856,-112.4682,19.69,31,4,1.79,US,1712096114


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df.drop(columns=['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
88,pacific grove,36.6177,-121.9166,73,US,
105,newman,37.3138,-121.0208,41,US,
119,portland,45.5234,-122.6762,41,US,
185,kerikeri,-35.2268,173.9474,55,NZ,
223,east london,-33.0153,27.9116,69,ZA,
245,kone,-21.0595,164.8658,62,NC,
315,ha'il,27.5219,41.6907,34,SA,
324,yeroham,30.9872,34.9314,51,IL,
376,thames,-37.1333,175.5333,55,NZ,
483,paulden,34.8856,-112.4682,31,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "categories": 'accommodation.hotel',
    "limit": 20,
    "apiKey": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"  # Update filter parameter
    params["bias"] = f"proximity:{lon},{lat}"  # Update bias parameter
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
pacific grove - nearest hotel: Pacific Grove Inn
newman - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
kerikeri - nearest hotel: Avalon Resort
east london - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
ha'il - nearest hotel: فندق كلاسيك
yeroham - nearest hotel: אירוס המדבר
thames - nearest hotel: No hotel found
paulden - nearest hotel: No hotel found
quepos - nearest hotel: Casa Feliz


,City,Lat,Lng,Humidity,Country,Hotel Name
88,pacific grove,36.6177,-121.9166,73,US,Pacific Grove Inn
105,newman,37.3138,-121.0208,41,US,No hotel found
119,portland,45.5234,-122.6762,41,US,the Hoxton
185,kerikeri,-35.2268,173.9474,55,NZ,Avalon Resort
223,east london,-33.0153,27.9116,69,ZA,No hotel found
245,kone,-21.0595,164.8658,62,NC,Pacifik Appartels
315,ha'il,27.5219,41.6907,34,SA,فندق كلاسيك
324,yeroham,30.9872,34.9314,51,IL,אירוס המדבר
376,thames,-37.1333,175.5333,55,NZ,No hotel found
483,paulden,34.8856,-112.4682,31,US,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    #size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ['Country', 'Hotel Name']
)

# Display the map

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)